- importing the libraries here...

In [1]:
import nltk
# nltk.download()
import pandas as pd
import spacy
# from spacy import displacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
!pip3 install fuzzywuzzy
!pip3 install python-Levenshtein

     |â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 51kB 2.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp36-cp36m-linux_x86_64.whl size=148895 sha256=b4526bcbe336bc8fdfdaaabc5ddb01a517867f7acaadf50805756e43ad9f2e3d
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [3]:
from fuzzywuzzy import fuzz

In [4]:
# !python -m spacy download en_core_web_lg

In [5]:
def  reading_file():
  
  tech = pd.read_csv("/content/Technology Summary Matching Project - technologies.csv")
  summary = pd.read_csv("/content/Testing-Sheet(23_02) - Sheet1.csv")
  output = pd.read_csv("/content/output - output.csv")

  print("Technology sheet", tech.head())
  print("\n\n Technology sheet data types\n", tech.dtypes)

  print("summary sheet", summary.head())
  print("\n\n summary sheet data types\n", summary.dtypes)

  print("output sheet", output.head())
  print("\n\n output sheet data types\n", output.dtypes)

  return tech, summary, output

In [6]:
tech, summary, output = reading_file()

Technology sheet          title
0      Lattice
1  WorkflowMax
2     InGenius
3      DealHub
4        Ebsta


 Technology sheet data types
 title    object
dtype: object
summary sheet                             id  ...                                     version_status
0  XOELhfz2TfdBzJQsZsYcew_0000  ...  {"contains":[],"previous_version":"9","current...
1  dAvAXCfTPxPXJ3K3jeGj4g_0000  ...  {"contains":[],"previous_version":"9","current...
2  E30RAmLIIdsOMP-fZCMCOQ_0000  ...  {"contains":[],"previous_version":"9","current...
3  BIL7L35xzb4vxuooMzQ3Ng_0000  ...  {"contains":[],"previous_version":"9","current...
4  DB7LnCzxCn93VYO2cG3ytA_0000  ...  {"contains":[],"previous_version":"9","current...

[5 rows x 19 columns]


 summary sheet data types
 id                            object
birth_date                    object
birth_date_fuzzy             float64
certifications                object
education                     object
emails                        object
experience           

In [7]:
output = output.drop("summaries", axis =1)

In [8]:

def dictionary_with_title_description(df):
  " This function will return dictionaries containing blog id, company names from title, and company name from description"
  
  company_names_description = {} # initialising empty dictionary for company names extracted from description
 
  for i in range(0, len(df)):
    doc = nlp(df.loc[i, "summaries"])
    
    for X in doc.ents:
      
      if X.label_ == 'ORG':

        company_names_description.setdefault(df.loc[i, "id"], [])
        company_names_description[df.loc[i, "id"]].append(X.text)
        # company_names_description{'blog_id' : list of companies found from ORG tag using spacy }
        
  print("Company names extracted from Summaries : \n",company_names_description)


  return company_names_description

In [9]:
# function call
company_names_description = dictionary_with_title_description(summary)

Company names extracted from Summaries : 
 {'DB7LnCzxCn93VYO2cG3ytA_0000': ['the university of colorado skaggs school of pharmacy and pharmaceutical sciences', 'the university of colorado skaggs school of pharmacy and pharmaceutical sciences student council', 'the student national pharmaceutical association', 'phi lambda sigma', 'the university of arizona', 'the university of arizona campus', 'mtm'], 'FnuiJCYmhgCYdhjrUgiXpg_0000': ['Neugene International'], 'bjKPZry3ODiQTOsoaJzO5Q_0000': ['Precious Eklad Ventures Ltd'], '0DxoDBIf4Gu73Qq9gnk-kA_0000': ['CIBC'], 'yWuSp-FD9DpqC4MUcKUHdQ_0000': ['fp&a', 'fp&a', 'fp&a', 'fp&a', 'fp&a', 'fp&a'], 'XimiDkHNmTI8QRdUw4IJGQ_0000': ['TEC Group International'], 'AgCVjEKXkCXY0hsvFdQefA_0000': ['Jaguar Land Rover'], 'uuPTGTLKCjSRq2wyKpYFPg_0000': ['samsung mobiles india'], '4wqNd8FCEFzKAhev70nL9g_0000': ['Zephyr Real Estate'], 'XBCv9B5Nfo4YmUkGlBosSQ_0000': ['Thermal & Wind Power Projects, Industrial Projects, Refineries & Gas Plants', 'Techno-Commer

In [10]:
def new_swapped_dict(df, company_names_description):
  company_names_description_new = {}
  
  for key, value in company_names_description.items():
    
    for j in range(0, len(value)):
      
      company_names_description_new[value[j]] = key
    
  final_dictionary = {}
  
  for key, val in company_names_description_new.items():
    for i in range(0 , len(tech)):
      if fuzz.ratio(key, tech.loc[i, "title"]) > 70:
        final_dictionary.setdefault(val,[])
        final_dictionary[val].append(tech.loc[i, "title"])
        # entries are of type : final_dictionary{'blog_id': company ids}

  return company_names_description_new, final_dictionary

In [11]:
# function call
company_names_description_new, final_dictionary = new_swapped_dict(summary, company_names_description)

In [12]:
tech["id"] = tech.index
tech.head()

,title,id
0,Lattice,0
1,WorkflowMax,1
2,InGenius,2
3,DealHub,3
4,Ebsta,4


In [13]:
try:
  
  for k, v in final_dictionary.items():
    # x = summary["summaries"].loc(summary["id"] == k)
    # print(x)
    new_row = {'id':k,'Technology_Names': final_dictionary[k]}
    output = output.append(new_row, ignore_index=True)

except Exception as e :
  print("\n\n", e)

In [14]:
# try:
#   for i in range(0, len(summary)):
#     new_row_2 = {'id':summary["id"][i], 'summaries':summary["summaries"][i], 'Technology_Names':final_dictionary[summary["id"][i]]}
#     output_2 = output.append(new_row, ignore_index=True)

# except Exception as e :
#   print("\n\n", e)
output.head()

,id,Technology_Names
0,FnuiJCYmhgCYdhjrUgiXpg_0000,"[YTZ International, ITN International, Retain ..."
1,0DxoDBIf4Gu73Qq9gnk-kA_0000,[CINC]
2,XimiDkHNmTI8QRdUw4IJGQ_0000,"[YTZ International, ITN International, Profile..."
3,XBCv9B5Nfo4YmUkGlBosSQ_0000,"[Technomedia, W3B Project Management, Azure Co..."
4,9qxO5Zf9PqKsuOjQl7wQRQ_0000,"[Microsoft FRx, Microsoft IIS, Microsoft.NET]"


In [15]:
output.to_csv("Output-NER_spacy(Test-23-02-profiles).csv", index = False)